<a href="https://colab.research.google.com/github/aminojagh/fast-ai/blob/main/NB5-Road-to-the-top.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Initial Setup

using fastkaggle to setup the competition (requires ~/kaggle/kaggle.json)

In [ ]:
kaggle_config_path = '/root/.config/kaggle'
!mkdir {kaggle_config_path}
from google.colab import files
files.upload(kaggle_config_path)
!chmod 600 /root/.config/kaggle/kaggle.json

In [ ]:
!pip install -Uq fastkaggle fastai

In [ ]:
from fastkaggle import setup_comp, iskaggle, push_notebook
from fastai.vision.all import (get_image_files, PILImage, set_seed,
                               ImageDataLoaders, Resize, aug_transforms,
                               vision_learner, error_rate, valley, slide)

from fastcore.parallel import parallel
import pandas as pd

In [ ]:
comp = 'paddy-disease-classification'
path = setup_comp(comp, install='fastai "timm>=0.6.2.dev0"')
print(path)
display(path.ls())

## Looaking at the Data

In [ ]:
trn_path = path/'train_images'
files = get_image_files(trn_path)
# img = PILImage.create(files[0])
# print(img.size)
# img.to_thumb(128)

In [ ]:
def f(o): return PILImage.create(o).size
sizes = parallel(f, files, n_workers=8)
pd.Series(sizes).value_counts()

In [ ]:
dls = ImageDataLoaders.from_folder(trn_path, valid_pct=0.2, seed=42,
    item_tfms=Resize(480, method='squish'),
    batch_tfms=aug_transforms(size=128, min_scale=0.75))

# dls.show_batch(max_n=6)

## Our first model

In [ ]:
learn = vision_learner(dls, 'resnet26d', metrics=error_rate, path='.').to_fp16()
learn.lr_find(suggest_funcs=(valley, slide))

In [ ]:
learn.fine_tune(3, 0.01)

## Submitting to Kaggle-I

In [ ]:
ss = pd.read_csv(path/'sample_submission.csv')
tst_files = get_image_files(path/'test_images').sorted()
tst_dl = dls.test_dl(tst_files)

probs,_,idxs = learn.get_preds(dl=tst_dl, with_decoded=True)
# print(idxs)
# print(dls.vocab)
mapping = dict(enumerate(dls.vocab))
results = pd.Series(idxs.numpy(), name="idxs").map(mapping)

ss['label'] = results
ss.to_csv('subm.csv', index=False)
# !head subm.csv

if not iskaggle:
    from kaggle import api
    api.competition_submit_cli('subm.csv', 'initial rn26d 128px', comp)
    # push_notebook('jhoward', 'first-steps-road-to-the-top-part-1',
    #               title='First Steps: Road to the Top, Part 1',
    #               file='first-steps-road-to-the-top-part-1.ipynb',
    #               competition=comp, private=False, gpu=True)

## Going faster

In [ ]:
from pathlib import Path
from fastai.vision.all import resize_images, ResizeMethod, PadMode
import numpy as np

In [ ]:
trn_path = Path('sml')
resize_images(path/'train_images', dest=trn_path, max_size=256, recurse=True)

In [ ]:
def train(trn_path, arch, item, batch, epochs=5):
    dls = ImageDataLoaders.from_folder(trn_path, seed=42, valid_pct=0.2, item_tfms=item, batch_tfms=batch)
    learn = vision_learner(dls, arch, metrics=error_rate).to_fp16()
    learn.fine_tune(epochs, 0.01)
    return learn

In [ ]:
# # our initial model
# learn = train(trn_path,
#               'resnet26d',
#               item=Resize(192),
#               batch=aug_transforms(size=128, min_scale=0.75))

## A ConvNeXt model

In [ ]:
arch = 'convnext_small_in22k'

# learn = train(trn_path,
#               arch,
#               item=Resize(192, method='squish'), # the default method is 'crop'
#               batch=aug_transforms(size=128, min_scale=0.75))

learn = train(trn_path,
              arch,
              item=Resize((256,192),
                          method=ResizeMethod.Pad, pad_mode=PadMode.Zeros),
              batch=aug_transforms(size=(171,128), min_scale=0.75))

## Test time augmentation

In [ ]:
valid = learn.dls.valid
preds,targs = learn.get_preds(dl=valid)
error_rate(preds, targs)

In [ ]:
tta_preds,_ = learn.tta(dl=valid)
error_rate(tta_preds, targs)

## Scaling up

In [ ]:
trn_path = path/'train_images'

learn = train(trn_path,
              arch,
              epochs=12,
              item=Resize((480, 360), method=ResizeMethod.Pad, pad_mode=PadMode.Zeros),
              batch=aug_transforms(size=(256,192), min_scale=0.75))

tta_preds,targs = learn.tta(dl=learn.dls.valid)
error_rate(tta_preds, targs)

## Submitting to Kaggle-II

In [ ]:
def submit_to_kaggle(sample_sub_file_path:Path,
                     test_images_path:Path,
                     is_kaggle:bool, tta:bool,
                     sub_title:str
                     ):

  ss = pd.read_csv(sample_sub_file_path)
  tst_files = get_image_files(test_images_path).sorted()
  tst_dl = learn.dls.test_dl(tst_files)

  if tta:
    preds,_ = learn.tta(dl=tst_dl)
    idxs = preds.argmax(dim=1)
  else:
    probs,_,idxs = learn.get_preds(dl=tst_dl, with_decoded=True)

  vocab = np.array(learn.dls.vocab)
  results = pd.Series(vocab[idxs], name="idxs")


  ss['label'] = results
  ss.to_csv('subm.csv', index=False)

  if not iskaggle:
      from kaggle import api
      api.competition_submit_cli('subm.csv', sub_title, comp)

In [ ]:
submit_to_kaggle(sample_sub_file_path = path/'sample_submission.csv',
                 test_images_path = path/'test_images',
                 is_kaggle = iskaggle, tta = True,
                 sub_title = 'convnext small 256x192 12 epochs tta')